In [1]:
# Input:  A set of kmers Motifs
# Output: Count(Motifs)
def Count(Motifs):
    count = {}  # initializing the count dictionary
    
    # Set k equal to the length of Motifs[0]
    k = len(Motifs[0])

    # Iterate over all nucleotides symbol and create a list of zeroes corresponding to count[symbol]
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(0)
    # Iterate over all elements symbol = Motifs[i][j] of the count matrix and add 1 to count[symbol][j]
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

In [2]:
# Input:  A list of kmers Motifs
# Output: the profile matrix of Motifs, as a dictionary of lists.
def Profile(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {}
    # insert your code here
    for symbol in "ATGC":
        profile[symbol] = []
        for j in range(k):
            profile[symbol].append(0)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            profile[symbol][j] += 1
    for symbol in "ACGT":
        for j in range(k):
            profile[symbol][j]=profile[symbol][j]/t
    return profile

In [4]:
# Input:  A set of kmers Motifs
# Output: A consensus string of Motifs.
def Consensus(Motifs):
    # insert your code here
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = "" #initialize an empty consensus string
    for j in range(k): #range through each column of the count matrix
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m: 
                m = count[symbol][j]
                frequentSymbol = symbol #adding the maximum element from column j at step j
        consensus += frequentSymbol
    return consensus

In [5]:
# Input:  A set of k-mers Motifs
# Output: The score of these k-mers.
def Score(Motifs):
    # Insert code here
    consensus = Consensus(Motifs)
    k = len(consensus)
    t = len(Motifs)
    score = 0
    for j in range(k):
        for i in range(t):
            if Motifs[i][j] != consensus[j]:
                score += 1
    return score

In [3]:
# Input:  String Text and profile matrix Profile
# Output: Pr(Text, Profile)
def Pr(Text, Profile):
    # insert your code here
    p = 1
    for i in range(len(Text)):
        p *= Profile[Text[i]][i]
    return p

In [5]:
def ProfileMostProbableKmer(text, k, profile):
    max_probability = -1.0
    most_probable_kmer = ""
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        probability = Pr(kmer, profile)
        if probability > max_probability:
            max_probability = probability
            most_probable_kmer = kmer
    return most_probable_kmer

In [7]:
# Input:  A list of kmers Dna, and integers k and t (where t is the number of kmers in Dna)
# Output: GreedyMotifSearch(Dna, k, t)
def GreedyMotifSearch(Dna, k, t):
    # Khởi tạo best_motifs là danh sách các k-mer đầu tiên từ mỗi chuỗi DNA trong Dna.
    n = len(Dna[0])
    best_motifs = [Dna[i][0:k] for i in range(t)]
    
    # Duyệt qua mỗi k-mer có thể có trong chuỗi DNA đầu tiên.
    for i in range(n - k + 1):
        motifs = [Dna[0][i:i+k]]
        
        # Với mỗi k-mer này, xây dựng một danh sách motifs bằng cách chọn k-mer có xác suất cao nhất từ mỗi chuỗi DNA còn lại.
        for j in range(1,t):
            profile = Profile(motifs[0:j])
            motifs.append(ProfileMostProbableKmer(Dna[j], k, profile))
        
        # Nếu score của motifs nhỏ hơn score của best_motifs, best_motifs sẽ được cập nhật thành motifs.
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
    
    # Trả về best_motifs, là danh sách các motif tốt nhất tìm thấy.
    return best_motifs

In [1]:
def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)  # Number of k-mers
    k = len(Motifs[0])  # Length of each k-mer
    profile = {}  # Initialize an empty dictionary for the profile↳

    # Compute the count matrix with pseudocounts
    count = CountWithPseudocounts(Motifs)
    # Calculate the probabilities for each base at each position
    for base in "ACGT":
        profile[base] = [count[base][i] / (t + 4) for i in range(k)]
    return profile

def CountWithPseudocounts(Motifs):
    t = len(Motifs)  # Number of k-mers
    k = len(Motifs[0])  # Length of each k-mer
    count = {}  # Initialize an empty dictionary for counts
    # Initialize the count matrix with pseudocounts
    for base in "ACGT":
        count[base] = [1] * k
    # Iterate through each position in the k-mers
    for i in range(k):
        for j in range(t):
            base = Motifs[j][i]
            count[base][i] += 1
    return count

In [2]:
def CountWithPseudocounts(Motifs):
    count = {symbol: [1] * len(Motifs[0]) for symbol in "ACGT"}
    for i in range(len(Motifs)):
        for j in range(len(Motifs[i])):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs) + 4
    count = CountWithPseudocounts(Motifs)
    profile = {symbol: [float(c) / t for c in count[symbol]] for symbol in "ACGT"}
    return profile
def ScoreWithPseudocounts(Motifs):
    count = CountWithPseudocounts(Motifs)
    score = 0
    for j in range(len(Motifs[0])):
        max_count = max(count[symbol][j] for symbol in "ACGT")
        score += sum(count[symbol][j] for symbol in "ACGT") - max_count
    return score

def ProfileMostProbablePattern(Text, k, profile):
    max_prob = -1
    most_probable = ""
    for i in range(len(Text) - k + 1):
        pattern = Text[i:i+k]
        prob = 1
        for j in range(k):
            prob *= profile[pattern[j]][j]
        if prob > max_prob:
            max_prob = prob
            most_probable = pattern
    return most_probable
def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    BestMotifs = [string[:k] for string in Dna]
    
    for i in range(len(Dna[0]) - k + 1):
        Motifs = [Dna[0][i:i+k]]
        
        for j in range(1, t):
            profile = ProfileWithPseudocounts(Motifs)
            motif = ProfileMostProbablePattern(Dna[j], k, profile)
            Motifs.append(motif)
        
        if ScoreWithPseudocounts(Motifs) < ScoreWithPseudocounts(BestMotifs):
            BestMotifs = Motifs
    
    return BestMotifs

# Test dataset
Dna = [
    "GGCGTTCAGGCA",
    "AAGAATCAGTCA",
    "CAAGGAGTTCGC",
    "CACGTCAATCAC",
    "CAATAATATTCG"
]
k = 3
t = 5

result = GreedyMotifSearchWithPseudocounts(Dna, k, t)

In [3]:
# Input:  A profile matrix Profile and a list of strings Dna
# Output: Motifs(Profile, Dna)
def Motifs(Profile, Dna):
    # insert your code here
    n = len(Dna[0])
    for key in Profile.keys():
        k = len(Profile[key])
    Motifs = []
    for i in range(len(Dna)):
        Motifs.append(ProfileMostProbablePattern(Dna[i], k, Profile))
    return Motifs
        
# Insert your ProfileMostProbablePattern(Text, k, Profile) and Pr(Pattern, Profile) functions here.
def Pr(Text, Profile):
    Pr = 1
    for i in range(len(Text)):
        Pr = Pr * Profile[Text[i]][i]
    return Pr

def ProfileMostProbablePattern(Text, k, profile):
    n = len(Text)
    dict = {}
    list1 = []
    for j in range(n - k + 1):
        ss = Text[j:j + k]
        prob = Pr(ss, profile)
        dict[ss] = prob
        list1.append(prob)
    maxx = max(list1)
    for key in dict.keys():
        if dict[key] == maxx:
            return key

In [4]:
# import Python's 'random' module here
import random 
# Input:  A list of strings Dna, and integers k and t
# Output: RandomMotifs(Dna, k, t)
# HINT:   You might not actually need to use t since t = len(Dna), but you may find it convenient
def RandomMotifs(Dna, k, t):
    # place your code here.
    random_motifs = []
    for i in range(t):
        start_index =random.randint(0, len(Dna[i]) - k)
        random_motifs.append(Dna[i][start_index:start_index + k])
    return random_motifs

In [5]:
# import the random package here
import random
# Input:  Positive integers k and t, followed by a list of strings Dna
# Output: RandomizedMotifSearch(Dna, k, t)
def RandomizedMotifSearch(Dna, k, t):
    # insert your code here
    M = RandomMotifs(Dna, k, t)
    BestMotifs = M
    while True:
        Profile = ProfileWithPseudocounts(M)
        M = Motifs(Profile, Dna)
        if Score(M) < Score(BestMotifs):
            BestMotifs = M
        else:
            return BestMotifs 

# Insert necessary subroutines here, including RandomMotifs(), ProfileWithPseudocounts(), Motifs(), Score(),
# and any subroutines that these functions need.
def ProfileWithPseudocounts(Motifs):

    t = len(Motifs)
    k = len(Motifs[0])
    profile = CountWithPseudocounts(Motifs) # output variable
    for symbol in profile:
        for kk in range(0,len(profile[symbol])):
            profile[symbol][kk] = profile[symbol][kk]/(len(Motifs) + 4)

    return profile


def CountWithPseudocounts(Motifs):
    count = {}
    for i in 'ACGT':
        count[i] = []
        for ii in range(len(Motifs[0])):
            count[i].append(1)
    for i in range(len(Motifs)):
        for j in range(len(Motifs[0])):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def Score(Motifs):


    count = 0
    L = Consensus(Motifs)
    for i in Motifs:
        for chr1, chr2 in zip(i,L):
            if chr1 != chr2:
                count += 1
    return count


def Consensus(Motifs):

    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol

    return consensus


def Count(Motifs):

    count = {}
    for i in 'ACGT':
        count[i] = []
        for ii in range(len(Motifs[0])):
            count[i].append(0)
    for i in range(len(Motifs)):
        for j in range(len(Motifs[0])):
            symbol = Motifs[i][j]
            count[symbol][j] += 1

    return count


def RandomMotifs(dna,k,t):

    kmm = []
    sc = []
    D = {}
    for i in range(0,len(dna)):
        km = []
        for kk in range(len(dna[i])-k+1):
            km += [dna[i][kk:kk+k]]
        D[i] = km
    for m in range(0,t):
        ran = random.randint(0,len(D[0])-1)
        kmm += [D[m][ran]]

    return kmm

def Motifs(pf,dna):

    k = len(pf['A'])
    D = []
    for i in range(0,len(dna)):
        km = []
        sc = []
        for kk in range(len(dna[i])-k+1):
            km += [dna[i][kk:kk+k]]
        for i in km:
            sc += [Pr(i,pf)]
        D += [km[sc.index(max(sc))]]

    return D


def Pr(Text, Profile):

    p = 1
    for i in range(0,len(Text)):
        p *= Profile[Text[i]][i]
    return p

In [6]:
# Input: A dictionary Probabilities, where keys are k-mers and values are the probabilities of these k-mers (which do not necessarily sum up to 1)
# Output: A normalized dictionary where the probability of each k-mer was divided by the sum of all k-mers' probabilities
def Normalize(Probabilities):
    # your code here
    total = sum(Probabilities.values())
    normalized = {kmer: prob / total for kmer, prob in Probabilities.items()}
    return normalized

In [ ]:
# first, import the random package
import random
# Input:  A dictionary Probabilities whose keys are k-mers and whose values are the probabilities of these kmers
# Output: A randomly chosen k-mer with respect to the values in Probabilities
def WeightedDie(Probabilities):
    kmer = '' # output variable
    # your code here
    # Generate a random number between 0 and 1
    p = random.uniform(0, 1)
    
    # Iterate through k-mers and their probabilities
    for kmer, probability in Probabilities.items():
        p -= probability
        if p <= 0:
            return kmer

In [ ]:
# first, import the random package
import random
# then, copy Pr, Normalize, and WeightedDie below this line
def Pr(Text, Profile):
    p = 1.0
    for i in range(len(Text)):
        symbol = Text[i]
        if symbol == 'A':
            row = 0
        elif symbol == 'C':
            row = 1
        elif symbol == 'G':
            row = 2
        else:
            row = 3
        p *= Profile[symbol][i]  # Use the symbol to index the correct row in the profile matrix
    return p
def Normalize(Probabilities):
    total = sum(Probabilities.values())
    for key in Probabilities:
        Probabilities[key] /= total
    return Probabilities
def WeightedDie(Probabilities):
    p = random.uniform(0, 1)
    cumulative_prob = 0
    for kmer, prob in Probabilities.items():
        cumulative_prob += prob
        if p < cumulative_prob:
            return kmer
    return None

# Input:  A string Text, a profile matrix Profile, and an integer k
# Output: ProfileGeneratedString(Text, profile, k)
def ProfileGeneratedString(Text, profile, k):
    # your code here
    n = len(Text)
    probabilities = {}
    for i in range(0, n - k + 1):
        kmer = Text[i:i+k]
        probabilities[kmer] = Pr(kmer, profile)
    probabilities = Normalize(probabilities)  # Normalize the probabilities
    return WeightedDie(probabilities)